In [1]:
import torch as T 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
from torchvision.transforms import ToTensor 
import numpy as np 
import os
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix
import pickle
import time 

In [6]:
def load_images_from_folder(folder, num_images=-1):
    images = []
    labels = []
    counter = 0 
    filenames = []
    for filename in os.listdir(folder):
        if counter >= num_images and num_images != -1:
            break
        img = cv2.imread(os.path.join(folder,filename), 0)
        try: 
            label = int(filename.split("_")[2])
            for i in range(len(img)):
                for j in range(len(img[i])):
                    img[i][j] = int(img[i][j] > 0.5 * 255)
            if (img is not None):
                images.append(img)
                labels.append(label)
                filenames.append(filename)
            counter += 1
        except:
            print(filename)
    return images, labels, filenames

In [5]:
def get_data(folder, num_images=-1):
    images, labels, _ = load_images_from_folder(folder, num_images)
    print(images[0][0:10, 0:10])
    X_t, X_test, y_t, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_t, y_t, test_size=0.25, random_state=1038)
    X_train = T.tensor(X_train)
    y_train = T.tensor(y_train)
    X_val = T.tensor(X_val)
    y_val = T.tensor(y_val)
    X_test = T.tensor(X_test)
    y_test = T.tensor(y_test)

    y_train = y_train.type(T.LongTensor)
    y_val = y_val.type(T.LongTensor)
    y_test = y_test.type(T.LongTensor)
    train_dataset = T.utils.data.TensorDataset(X_train, y_train)
    val_dataset = T.utils.data.TensorDataset(X_val, y_val)
    test_dataset = T.utils.data.TensorDataset(X_test, y_test)
    return train_dataset, val_dataset, test_dataset
train_dataset, val_dataset, test_dataset = get_data("endpt_dataset/window_images")
with open('train_endptdataset.pickle', 'wb') as f:
    pickle.dump(train_dataset, f)
with open('val_endptdataset.pickle', 'wb') as f:
    pickle.dump(val_dataset, f)
with open('test_endptdataset.pickle', 'wb') as f:
    pickle.dump(test_dataset, f)

[[1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]]


In [2]:
class CNN(nn.Module):
    def __init__(self, lr, epochs, input_dims, batch_size, name, best_name, chkpt_dir, dataset_prefix, num_classes=2):
        super(CNN, self).__init__()
        self.chkpt_dir = chkpt_dir
        self.checkpoint_file = os.path.join(self.chkpt_dir, name)
        self.best_checkpoint_file = os.path.join(self.chkpt_dir, best_name)
        self.epochs = epochs
        self.lr = lr
        self.input_dims = input_dims
        self.batch_size = batch_size 
        self.num_classes = num_classes
        self.loss_history = []
        self.acc_history = []
        self.skipped_points = []
        self.val_history = [0, 0, 0]
        self.device = T.device("cpu")
        
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, 3)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 32, 3)
        self.bn3 = nn.BatchNorm2d(32)
        self.maxpool1 = nn.MaxPool2d(2)
        self.conv4 = nn.Conv2d(32, 64, 3)
        self.bn4 = nn.BatchNorm2d(64)
        self.conv5 = nn.Conv2d(64, 64, 3)
        self.bn5 = nn.BatchNorm2d(64)
        self.conv6 = nn.Conv2d(64, 64, 3)
        self.bn6 = nn.BatchNorm2d(64)
        self.maxpool2 = nn.MaxPool2d(2)

        fc_dims = self.calc_fc_dims()
        self.fc1 = nn.Linear(fc_dims, 32)
        self.fc2 = nn.Linear(32, 16) 
        self.fc3 = nn.Linear(16, self.num_classes)

        self.optimizer = optim.Adam(self.parameters(), lr=self.lr) # the self.parameters() comes from nn.Module 

        self.loss = nn.CrossEntropyLoss() # since we have more than 2 classes cross entropy is best otherwise with 2 classes we might be able to use binary loss
        
        self.to(self.device)
        self.get_data("train" + dataset_prefix, "val" + dataset_prefix, "test" + dataset_prefix)

    def calc_fc_dims(self):
        batch_data = T.zeros((1, 1, 25, 25)) # 4-tensor of 0s and we plug it into the layer and see what comes out 
        batch_data = self.conv1(batch_data)
        # batch_data = self.bn1(batch_data) # batch norm layer does not change dimesnionality
        batch_data = self.conv2(batch_data)
        # batch_data = self.bn2(batch_data)
        batch_data = self.conv3(batch_data)
        # batch_data = self.bn3(batch_data)
        batch_data = self.maxpool1(batch_data)
        batch_data = self.conv4(batch_data)
        batch_data = self.conv5(batch_data)
        batch_data = self.conv6(batch_data)
        batch_data = self.maxpool2(batch_data)

        return int(np.prod(batch_data.size())) # this will give us the input dimesniosn 

    def forward(self, batch_data): # NOTE: we can kinda combine this with calc_fc_dims() 
        batch_data = T.tensor(batch_data).to(self.device) # lower case tensor() preserves the datatype while Tensor() changes the datatype to some default datatype 
        # we do a to(self.device) in order to make sure it is not a cuda tensor 
        try: 
            batch_data = T.reshape(batch_data, self.input_dims)
        except:
            pass
        batch_data = batch_data.type(T.FloatTensor)
#             print(batch_data.size(), batch_data.type(), batch_data)
# 		print(batch_data.type())
        batch_data = self.conv1(batch_data)
        batch_data = self.bn1(batch_data) # debate about whether to do batch norm before or after relu but this works fine for this one especially since relu is a noncommutative operation with respect to things like addition
        batch_data = F.relu(batch_data)

        batch_data = self.conv2(batch_data)
        batch_data = self.bn2(batch_data)
        batch_data = F.relu(batch_data)

        batch_data = self.conv3(batch_data)
        batch_data = self.bn3(batch_data)
        batch_data = F.relu(batch_data)

        batch_data = self.maxpool1(batch_data)

        batch_data = self.conv4(batch_data)
        batch_data = self.bn4(batch_data)
        batch_data = F.relu(batch_data) 

        batch_data = self.conv5(batch_data)
        batch_data = self.bn5(batch_data)
        batch_data = F.relu(batch_data)

        batch_data = self.conv6(batch_data)
        batch_data = self.bn6(batch_data)
        batch_data = F.relu(batch_data)

        batch_data = self.maxpool2(batch_data)

        batch_data = batch_data.view(batch_data.size()[0], -1)

        batch_data = self.fc1(batch_data)
        batch_data = self.fc2(batch_data)
        classes = self.fc3(batch_data)
        # note that we are not doing another activation after this since the linear cross entropy loss performs a softmax activation on it already 

        return classes
        
    def get_data(self, train_filename, val_filename, test_filename):

        with open(train_filename, 'rb') as f:
            train_dataset = pickle.load(f)
        with open(val_filename, 'rb') as f:
            val_dataset = pickle.load(f)
        with open(test_filename, 'rb') as f:
            test_dataset = pickle.load(f)
        
# 		mnist_train_data = MNIST("mnist", train=True, download=True, 
# 								transform=ToTensor())
        self.train_data_loader = T.utils.data.DataLoader(train_dataset, 
                                    batch_size=self.batch_size, shuffle=True, # always want to shuffle the in case it was not preshuffled so that we get actual learning 
                                    num_workers=3) # this part is just so that the computer can split up the task so make it less than 4 for a mac 
        self.val_data_loader = T.utils.data.DataLoader(val_dataset, 
                                    batch_size=1, shuffle=True, 
                                    num_workers=3)
# 		mnist_test_data = MNIST("mnist", train=False, download=True, 
# 								transform=ToTensor())
        self.test_data_loader = T.utils.data.DataLoader(test_dataset, 
                                    batch_size=self.batch_size, shuffle=True, # always want to shuffle the in case it was not preshuffled so that we get actual learning 
                                    num_workers=3) 

    def _val(self):
        y_true = []
        y_pred = []
        for j, (input, label) in enumerate(self.val_data_loader): 
            input = T.reshape(input, (1,1,25,25))
#             print(input.type())
            prediction = self.forward(input)
            prediction = F.softmax(prediction, dim=1)
            classes = T.argmax(prediction, dim=1)
            y_pred.append(classes.item())
            y_true.append(label.item())
#         print("y_true is: \n", y_true[:10], "\n ---------------------------- \n ", "y_pred is: \n", y_pred[:10])
        conf_matrix = confusion_matrix(y_true, y_pred, labels=[0,1])
        print("The confusion matrix is: \n", conf_matrix)
        self.val_history.append(f1_score(y_true, y_pred, zero_division=1))
        return conf_matrix[0][0] + conf_matrix[1][1]/(np.sum(conf_matrix))
        
        
    def _train(self):
        self.train() # this is important if you are using pytorch with batch norm (it only switches the neural net to a train mode where it remembers the batch norm statistics for training thus only do this with batch norm)
        for i in range(self.epochs): # iteration over the full dataset (we have 60,000 in training set, 10,000 in the test set) so we want to iterate over it many many times 
            ep_loss = 0
            ep_acc = [] # this is epoch accuracy 
            counter = 0
            for j, (input, label) in enumerate(self.train_data_loader): # the default format is an integer and a tuple with an input and an actual label
#                 print(input.type())
                self.optimizer.zero_grad() # remember to always zero the gradient before your training as otherwise it will remember stuff from the last cycle 
                label = label.to(self.device)
# 				print(input.size(), label)
# 				print(input.type(), label.type())
                if input.size()[0] != 8:
                    print("I am passing")
                    counter += 1
                else:
                    prediction = self.forward(input)
                    loss = self.loss(prediction, label)
                    prediction = F.softmax(prediction, dim=1) # the softmax is so that we get a probabilities over the classes 
                    classes = T.argmax(prediction, dim=1)
                    wrong = T.where(classes != label, T.tensor([1.]).to(self.device), T.tensor([0.]).to(self.device)) # this looks at when the labels are not correct and marsk those with a 1
                    acc = 1 - T.sum(wrong) / self.batch_size
                    
                    ep_acc.append(acc.item()) # acc is a tensor so we look at the item in the tensor 
                    self.acc_history.append(acc.item())
                    ep_loss += loss.item()
                    loss.backward() # this calculates the gradient and is VERY IMPORTANT 
                    self.optimizer.step() # this uses the optimizer to adjust the weights ALSO VERY IMPORTANT 

                    if (j % 1000 == 0):
                        print("Epoch ", i, "Data Point ", j, "total loss %.3f" % ep_loss, "accuracy %.3f" % np.mean(ep_acc))

            self._val()
            print("Finished Epoch ", i, "total loss %.3f" % ep_loss, "accuracy %.3f" % np.mean(ep_acc), 
                  "validation f1 %.3f" % self.val_history[-1])
            print(self.val_history)
            self.skipped_points.append(counter)
            if counter:
                print("Number of skipped points is ", counter)
            self.loss_history.append(ep_loss)
#             if (i % 1 == 0):
            if self.val_history[-1] >= self.val_history[-2]:
                self.save_checkpoint(best=self.val_history[-1] == max(self.val_history))
#             if self.val_history[-1] < self.val_history[-2] and self.val_history[-2] < self.val_history[-3]:
#                 break
    def _test(self):
        
        # self.test() # this is important if you are using pytorch with batch norm so now we can run it in test mode
        ep_loss = 0
        ep_acc = [] # this is epoch accuracy 
        counter = 0
        y_true = []
        y_pred = []
        for j, (input, label) in enumerate(self.test_data_loader): # the default format is an integer and a tuple with an input and an actual label
            label = label.to(self.device)
            if input.size()[0] != 8:
#                 print("I am passing")
                counter += 1
            else:  
                prediction = self.forward(input)
                loss = self.loss(prediction, label)
                prediction = F.softmax(prediction, dim=1) # the softmax is so that we get a probabilities over the classes 
                classes = T.argmax(prediction, dim=1)
                wrong = T.where(classes != label, T.tensor([1.]).to(self.device), T.tensor([0.]).to(self.device)) # this looks at when the labels are not correct and marsk those with a 1
                acc = 1 - T.sum(wrong) / self.batch_size
                for i in range(self.batch_size):
                    y_true.append(label[i].item())
                    y_pred.append(classes[i].item())
                ep_acc.append(acc.item()) # acc is a tensor so we look at the item in the tensor 
                ep_loss += loss.item()
        print("The confusion matrix is: \n", confusion_matrix(y_true, y_pred, labels=[0,1]))
        print(f"The f1 score is {f1_score(y_true, y_pred, zero_division=1)}")

        print("Total loss %.3f" % ep_loss, "accuracy %.3f" % np.mean(ep_acc))
        if counter:
            print("Number of skipped points is ", counter)
        
        return ep_acc

    def save_checkpoint(self, filename=None, best=False):
        filename = self.checkpoint_file if filename is None else filename
        print('... saving checkpoint ...')
        T.save(self.state_dict(), filename)
        if best:
            T.save(self.state_dict(), self.best_checkpoint_file)

    def load_checkpoint(self, filename=None):
        filename = self.best_checkpoint_file if filename is None else filename
        print('... loading checkpoint ...')
        self.load_state_dict(T.load(filename))

        




In [3]:
network = CNN(lr=0.001, input_dims=(8,1,25,25), batch_size=8, epochs=20, name='CNN_V5', best_name='CNN_best_V5', chkpt_dir='cnn_history/endpt_cnn', dataset_prefix = "_endptdataset.pickle")

In [4]:
positive_counter = 0
counter = 0
for filename in os.listdir("endpt_dataset/window_images"):
    positive_counter += int(filename.split("_")[2])
    counter += 1
print(positive_counter, counter, positive_counter/counter)
        

13787 379852 0.036295715173278016


In [8]:
start_time = time.time()
# network.load_checkpoint(filename="cnn_history/CNN_best_V3")
network._train()

end_time = time.time()
print("The amount of time for training is: ", end_time - start_time)

/Users/sidhartkrishnan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Epoch  0 Data Point  0 total loss 0.678 accuracy 0.750
Epoch  0 Data Point  1000 total loss 129.276 accuracy 0.963
Epoch  0 Data Point  2000 total loss 230.775 accuracy 0.966
Epoch  0 Data Point  3000 total loss 313.325 accuracy 0.968
Epoch  0 Data Point  4000 total loss 371.199 accuracy 0.972
Epoch  0 Data Point  5000 total loss 423.802 accuracy 0.974
Epoch  0 Data Point  6000 total loss 471.928 accuracy 0.976
Epoch  0 Data Point  7000 total loss 527.299 accuracy 0.977
Epoch  0 Data Point  8000 total loss 573.084 accuracy 0.978
Epoch  0 Data Point  9000 total loss 614.376 accuracy 0.979
Epoch  0 Data Point  10000 total loss 662.854 accuracy 0.980
Epoch  0 Data Point  11000 total loss 700.621 accuracy 0.980
Epoch  0 Data Point  12000 total loss 739.191 accuracy 0.981
Epoch  0 Data Point  13000 total loss 771.975 accuracy 0.982
Epoch  0 Data Point  14000 total loss 804.808 accuracy 0.982
Epoch  0 Data Point  15000 total loss 838.009 accuracy 0.983
Epoch  0 Data Point  16000 total loss 8

Epoch  4 Data Point  4000 total loss 66.681 accuracy 0.994
Epoch  4 Data Point  5000 total loss 85.097 accuracy 0.994
Epoch  4 Data Point  6000 total loss 102.928 accuracy 0.994
Epoch  4 Data Point  7000 total loss 122.412 accuracy 0.994
Epoch  4 Data Point  8000 total loss 139.302 accuracy 0.994
Epoch  4 Data Point  9000 total loss 159.047 accuracy 0.994
Epoch  4 Data Point  10000 total loss 177.315 accuracy 0.994
Epoch  4 Data Point  11000 total loss 198.044 accuracy 0.994
Epoch  4 Data Point  12000 total loss 218.852 accuracy 0.994
Epoch  4 Data Point  13000 total loss 235.261 accuracy 0.994
Epoch  4 Data Point  14000 total loss 250.489 accuracy 0.994
Epoch  4 Data Point  15000 total loss 265.713 accuracy 0.994
Epoch  4 Data Point  16000 total loss 285.952 accuracy 0.994
Epoch  4 Data Point  17000 total loss 307.840 accuracy 0.994
Epoch  4 Data Point  18000 total loss 325.835 accuracy 0.994
Epoch  4 Data Point  19000 total loss 347.018 accuracy 0.994
Epoch  4 Data Point  20000 total

Epoch  8 Data Point  2000 total loss 29.972 accuracy 0.994
Epoch  8 Data Point  3000 total loss 42.981 accuracy 0.995
Epoch  8 Data Point  4000 total loss 57.310 accuracy 0.995
Epoch  8 Data Point  5000 total loss 73.537 accuracy 0.994
Epoch  8 Data Point  6000 total loss 86.718 accuracy 0.995
Epoch  8 Data Point  7000 total loss 103.527 accuracy 0.994
Epoch  8 Data Point  8000 total loss 122.126 accuracy 0.994
Epoch  8 Data Point  9000 total loss 136.040 accuracy 0.994
Epoch  8 Data Point  10000 total loss 150.266 accuracy 0.994
Epoch  8 Data Point  11000 total loss 164.299 accuracy 0.995
Epoch  8 Data Point  12000 total loss 179.033 accuracy 0.995
Epoch  8 Data Point  13000 total loss 194.734 accuracy 0.995
Epoch  8 Data Point  14000 total loss 204.494 accuracy 0.995
Epoch  8 Data Point  15000 total loss 218.197 accuracy 0.995
Epoch  8 Data Point  16000 total loss 230.096 accuracy 0.995
Epoch  8 Data Point  17000 total loss 244.806 accuracy 0.995
Epoch  8 Data Point  18000 total loss

Epoch  12 Data Point  0 total loss 0.001 accuracy 1.000
Epoch  12 Data Point  1000 total loss 9.848 accuracy 0.997
Epoch  12 Data Point  2000 total loss 21.774 accuracy 0.996
Epoch  12 Data Point  3000 total loss 33.887 accuracy 0.996
Epoch  12 Data Point  4000 total loss 46.890 accuracy 0.996
Epoch  12 Data Point  5000 total loss 57.930 accuracy 0.996
Epoch  12 Data Point  6000 total loss 69.960 accuracy 0.996
Epoch  12 Data Point  7000 total loss 80.412 accuracy 0.996
Epoch  12 Data Point  8000 total loss 94.280 accuracy 0.996
Epoch  12 Data Point  9000 total loss 106.016 accuracy 0.996
Epoch  12 Data Point  10000 total loss 117.744 accuracy 0.996
Epoch  12 Data Point  11000 total loss 129.893 accuracy 0.996
Epoch  12 Data Point  12000 total loss 142.177 accuracy 0.996
Epoch  12 Data Point  13000 total loss 157.108 accuracy 0.996
Epoch  12 Data Point  14000 total loss 167.830 accuracy 0.996
Epoch  12 Data Point  15000 total loss 179.160 accuracy 0.996
Epoch  12 Data Point  16000 tota

Epoch  15 Data Point  24000 total loss 243.335 accuracy 0.996
Epoch  15 Data Point  25000 total loss 255.530 accuracy 0.996
Epoch  15 Data Point  26000 total loss 263.619 accuracy 0.996
Epoch  15 Data Point  27000 total loss 273.416 accuracy 0.996
Epoch  15 Data Point  28000 total loss 283.532 accuracy 0.996
I am passing
The confusion matrix is: 
 [[72733   514]
 [ 2415   309]]
Finished Epoch  15 total loss 290.035 accuracy 0.996 validation f1 0.174
[0, 0, 0, 0.10595115239078087, 0.24969097651421504, 0.17528830313014826, 0.09192200557103064, 0.18015665796344646, 0.24906367041198504, 0.28917853895119733, 0.16381882289123947, 0.12780194044831047, 0.22668112798264645, 0.17650639074863053, 0.1924910607866508, 0.1988416988416988, 0.14394189501485638, 0.125, 0.17423174513673526]
Number of skipped points is  1
... saving checkpoint ...
Epoch  16 Data Point  0 total loss 0.010 accuracy 1.000
Epoch  16 Data Point  1000 total loss 8.898 accuracy 0.996
Epoch  16 Data Point  2000 total loss 17.584

Epoch  19 Data Point  6000 total loss 45.818 accuracy 0.997
Epoch  19 Data Point  7000 total loss 57.507 accuracy 0.997
Epoch  19 Data Point  8000 total loss 63.965 accuracy 0.997
Epoch  19 Data Point  9000 total loss 70.779 accuracy 0.997
Epoch  19 Data Point  10000 total loss 78.179 accuracy 0.997
Epoch  19 Data Point  11000 total loss 88.130 accuracy 0.997
Epoch  19 Data Point  12000 total loss 95.633 accuracy 0.997
Epoch  19 Data Point  13000 total loss 103.132 accuracy 0.997
Epoch  19 Data Point  14000 total loss 110.533 accuracy 0.997
Epoch  19 Data Point  15000 total loss 119.738 accuracy 0.997
Epoch  19 Data Point  16000 total loss 126.868 accuracy 0.997
Epoch  19 Data Point  17000 total loss 135.069 accuracy 0.997
Epoch  19 Data Point  18000 total loss 145.976 accuracy 0.997
Epoch  19 Data Point  19000 total loss 155.458 accuracy 0.997
Epoch  19 Data Point  20000 total loss 164.183 accuracy 0.997
Epoch  19 Data Point  21000 total loss 173.032 accuracy 0.997
Epoch  19 Data Poin

In [5]:
network.load_checkpoint(filename="cnn_history/endpt_cnn/CNN_best_V5")
network._val()
print(network.val_history[-1])

... loading checkpoint ...


/Users/sidhartkrishnan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


The confusion matrix is: 
 [[73149    98]
 [ 2247   477]]
0.28917853895119733


In [6]:
network.load_checkpoint(filename="cnn_history/endpt_cnn/CNN_best_V5")
ep_acc = network._test()

... loading checkpoint ...


/Users/sidhartkrishnan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


The confusion matrix is: 
 [[72858   343]
 [  135  2632]]
The f1 score is 0.9167537443399513
Total loss 179.281 accuracy 0.994
Number of skipped points is  1


In [7]:
np.mean(ep_acc)

0.9937078770008425

In [ ]:
len(network.acc_history)

In [7]:
# num_images = 10000
images, labels, filenames = load_images_from_folder("test_mini_images")
network.load_checkpoint(filename="cnn_history/endpt_cnn/CNN_best_V5")
print(len(images), len(labels))

... loading checkpoint ...
25635 25635


In [8]:
images = T.tensor(images, dtype = T.float)
images = T.reshape(images, (len(images),1,25,25))
print(images.size())

torch.Size([25635, 1, 25, 25])


In [17]:
# network.load_checkpoint(filename="cnn_history/CNN_best_V4")
prediction = network.forward(images)
prediction = F.softmax(prediction, dim=1) # the softmax is so that we get a probabilities over the classes 
print(prediction.size())
classes = T.argmax(prediction, dim=1)
# classes = [int((p[1] > 0.8).item()) for p in prediction]
print("Classes is ", len(classes))
counter = 0
for i in range(len(labels)):
    if labels[i] != classes[i]:
#         print(classes[i], i)
        counter += 1
print(counter)

/Users/sidhartkrishnan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


torch.Size([25635, 2])
Classes is  25635
205


In [18]:
def show(image):
    plt.figure()
    plt.imshow(image)

img = cv2.imread('test_images/test4.jpeg')
print(sum(classes))
for i in range(len(images)):
    if classes[i]:
#         print(filenames[i])
        coords = [int(filenames[i].split("_")[0]), int(filenames[i].split("_")[1])]
        for x in range(coords[0], coords[0] + 25):
#         for x in range(coords[0] - 12, coords[0] + 38):
            for y in [coords[1], coords[1] + 25]:
#             for y in [coords[1] - 12, coords[1] + 38]:
                img[x][y] = (255,0,0)
        for y in range(coords[1], coords[1] + 25):
#         for y in range(coords[1] - 12, coords[1] + 38):
            for x in [coords[0], coords[0] + 25]:
#             for x in [coords[0] - 12, coords[0] + 38]:
                img[x][y] = (255,0,0)

cv2.imwrite('boxed_images/boxed5.jpg', img)

tensor(205)


True

In [ ]:
# images, labels, filenames = load_images_from_folder("final_dataset", num_images=10000)
# for img in images:
#     for i in range(len(img)):
#         for j in range(len(img[i])):
#             if img[i][j] > 25 and img[i][j] < 230:
#                 print(img[i][j])